In [1]:
import pandas as pd
import numpy as np
import gzip
import os
import shutil

from ast import literal_eval
import json
from datetime import datetime
from sqlalchemy import create_engine

In [2]:
!ls


sample_data


In [20]:
for i in os.listdir():
  if i.endswith('.json'):
    os.remove(i)
  if i.endswith('.json.gz'):
    with gzip.open(i, 'rb') as f_in:
          with open(i.replace('.gz',''), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [21]:
from sqlalchemy.sql.expression import column
import math

user=pd.read_json('users.json',lines=True)
user.loc[:,'userId']=pd.json_normalize(user._id)
user.drop('_id',axis=1,inplace=True)
user.createdDate=pd.json_normalize(user.createdDate)
user.lastLogin=pd.json_normalize(user.lastLogin)

receipt=pd.read_json('receipts.json',lines=True)
receipt.loc[:,'receiptId']=pd.json_normalize(receipt._id)
receipt.drop('_id',axis=1,inplace=True)
receipt.createDate=pd.json_normalize(receipt.createDate)
receipt.dateScanned=pd.json_normalize(receipt.dateScanned)
receipt.finishedDate=pd.json_normalize(receipt.finishedDate)
receipt.modifyDate=pd.json_normalize(receipt.modifyDate)
receipt.pointsAwardedDate=pd.json_normalize(receipt.pointsAwardedDate)
receipt.purchaseDate=pd.json_normalize(receipt.purchaseDate)


brands=pd.read_json('brands.json',lines=True)
brands.loc[:,'brandsId']=pd.json_normalize(brands._id)
brands.drop('_id',axis=1,inplace=True)

In [22]:
rril=receipt[['receiptId','rewardsReceiptItemList']]
receipt.drop('rewardsReceiptItemList',axis=1,inplace=True)

In [23]:
barcode=[]
review=[]

def func1(y,x):
  global barcode
  global review
  if type(x) != float and x!=np.nan:
    for i in x:
      if 'barcode' in i.keys():
        i.update({'receiptId':y})
        barcode.append(i)
      else:
        i.update({'receiptId':y})
        review.append(i)



In [24]:
rril.apply(lambda x : func1(x.receiptId,x.rewardsReceiptItemList),axis=1)
barcodeScan=pd.json_normalize(barcode)
reviewNeeded=pd.json_normalize(review)

In [25]:
user.head(2)

,active,createdDate,lastLogin,role,signUpSource,state,userId
0,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052
1,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052


In [26]:
brands.head(2)

,barcode,category,categoryCode,cpg,name,topBrand,brandCode,brandsId
0,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,601ac115be37ce2ead437551
1,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,601c5460be37ce2ead43755f


In [27]:
receipt.head(2)

,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,receiptId
0,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575
1,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b


In [28]:
barcodeScan.head(2)

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,competitorRewardsGroup,metabriteCampaignId,originalReceiptItemText,priceAfterCoupon
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
reviewNeeded.head(2)

,needsFetchReview,partnerItemId,preventTargetGapPoints,userFlaggedBarcode,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity,receiptId,brandCode,competitorRewardsGroup,...,originalReceiptItemText,quantityPurchased,needsFetchReviewReason,userFlaggedDescription,deleted,priceAfterCoupon,competitiveProduct,rewardsProductPartnerId,metabriteCampaignId,rewardsGroup
0,False,1,True,4011,True,26.00,3.0,5ff1e1f10a720f052300057a,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1009,NaN,NaN,NaN,NaN,NaN,5ff1e1cd0a720f052300056f,MISSION,TACO BELL TACO SHELLS,...,MSSN TORTLLA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data quality Issue 1 -
# Missing value are present in user,receipt,brands which need to be addressed

In [30]:
user.isnull().sum(axis=0)


active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
userId           0
dtype: int64

In [31]:
receipt.isnull().sum(axis=0)


bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
receiptId                    0
dtype: int64

In [32]:
brands.isnull().sum(axis=0)



barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
brandsId          0
dtype: int64

# Data quality Issue 2 -
# There are duplicated record in user file.

In [34]:
user[user.duplicated()].head(3)

,active,createdDate,lastLogin,role,signUpSource,state,userId
1,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052
2,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052
4,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052


In [35]:
user.drop_duplicates(inplace=True)

# Data quality Issue 3 -
# As the data dictionary the role column should have constant value as consumer but found additional value fetch-staff in the column.

In [46]:
user.role.unique()

array(['consumer', 'fetch-staff'], dtype=object)

# Data quality Issue 4 -
# Certain userID mentioned in receipt file/table cannot be found in user file/table.

In [56]:
receipt.join(user,how='left',lsuffix='_receipt',rsuffix='_user')[['userId_receipt','userId_user']]

,userId_receipt,userId_user
0,5ff1e1eacfcf6c399c274ae6,5ff1e194b6a9d73a3a9f1052
1,5ff1e194b6a9d73a3a9f1052,NaN
2,5ff1e1f1cfcf6c399c274b0b,NaN
3,5ff1e1eacfcf6c399c274ae6,5ff1e1eacfcf6c399c274ae6
4,5ff1e194b6a9d73a3a9f1052,NaN
...,...,...
1114,5fc961c3b8cfca11a077dd33,NaN
1115,5fc961c3b8cfca11a077dd33,NaN
1116,5fc961c3b8cfca11a077dd33,NaN
1117,5fc961c3b8cfca11a077dd33,NaN


# Creating csv file to load in sql

In [ ]:
user.to_csv('user.csv',index=False,header=True)
receipt.to_csv('receipt.csv',index=False,header=True)
brands.to_csv('brands.csv',index=False,header=True)
barcodeScan.to_csv('barcodeScan.csv',index=True,header=True)
reviewNeeded.to_csv('reviewNeeded.csv',index=True,header=True)